# Lab 6: Cell Barcode Detection & UMI Analysis

**Module:** 6 - Cell Barcode Detection & UMI Deduplication  
**Duration:** 45-60 minutes

## Objectives
- Visualize barcode rank plots (knee plots)
- Understand how cells are distinguished from empty droplets
- Explore UMI distributions


In [ ]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

sc.settings.verbosity = 3
sc.set_figure_params(dpi=100)


In [ ]:
# Load PBMC dataset
adata = sc.datasets.pbmc3k()
sc.pp.calculate_qc_metrics(adata, inplace=True)
adata


## 1. Simulate a Barcode Rank Plot

In real pipelines, you'd have UMI counts for ALL barcodes (including empty droplets).
Here we'll simulate what that looks like.


In [ ]:
# Get real cell UMI counts
real_cell_counts = adata.obs['total_counts'].values

# Simulate empty droplet background (ambient RNA)
np.random.seed(42)
n_empty = 50000  # Many more empty droplets than cells
empty_counts = np.random.exponential(scale=50, size=n_empty)

# Combine and sort
all_counts = np.concatenate([real_cell_counts, empty_counts])
sorted_counts = np.sort(all_counts)[::-1]  # Descending order


In [ ]:
# Create barcode rank plot (knee plot)
fig, ax = plt.subplots(figsize=(10, 6))

ranks = np.arange(1, len(sorted_counts) + 1)
ax.scatter(ranks, sorted_counts, s=1, alpha=0.5)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Barcode Rank')
ax.set_ylabel('UMI Count')
ax.set_title('Barcode Rank Plot (Knee Plot)')

# Mark approximate knee
ax.axvline(x=len(real_cell_counts), color='red', linestyle='--', 
           label=f'~{len(real_cell_counts)} cells')
ax.legend()

plt.tight_layout()
plt.show()


## 2. The Knee/Inflection Point

The "knee" separates:
- **Above knee**: Real cells with high UMI counts
- **Below knee**: Empty droplets with ambient RNA


In [ ]:
# Calculate where the transition happens
n_real_cells = len(real_cell_counts)
min_real_cell_count = real_cell_counts.min()
max_empty_count = empty_counts.max()

print(f"Number of real cells: {n_real_cells}")
print(f"Minimum UMI in real cells: {min_real_cell_count:.0f}")
print(f"Maximum UMI in empty droplets: {max_empty_count:.0f}")
print(f"\nThe challenge: some barcodes are ambiguous!")


## Exercise Questions

1. Why is the relationship between genes and UMIs not perfectly linear?
2. What would happen if you set the UMI threshold too high?
3. What would happen if you set the UMI threshold too low?
4. Why is EmptyDrops better than a simple threshold for some datasets?


In [ ]:
# Your answers here

